## № 3

Consider trading a universe of $n$ stocks over $T$ days, with noisy predictions for the stock returns $p_{ti}$ available in `pred.npy` (the rows are days, the data start with day 0). Assuming that we have the position evolving as $\pi_t$ ($\pi_t$ at each $t$ is a vector of $n$ components), expected risk-adjusted gain $G$ reads
$$
G = \sum_{t} \left[p_{t} \cdot\pi_{t} - \pi_{t}^T \cdot \Omega \cdot \pi_{t} - \gamma\sum_i |\pi_{t,i} - \pi_{t-1,i}|\right]
\tag{12}
$$
and has to be maximized over $\pi_t$ (pick $\gamma=0.01$). The matrix $\Omega$ is in the `cov.npy` file.

In [3]:
import numpy as np

In [4]:
pred = np.load('pred.npy')
Omega = np.load('cov.npy')

### Part 1

Start with $t=1$ ($\pi_0=0$ by definition). At this moment, you have access to $p_1$ and have to maximize:
$$
p_1\pi_1 - \pi_1^T\Omega\pi_1 - \gamma \sum_i |\pi_{1, i} - 0|\longrightarrow \max
$$
over $\pi_1$. Show that this is a concave function of $\pi_1$ and maximize it using $\texttt{cvxpy}$.

In [5]:
import cvxpy as cp

In [10]:
gamma = 0.01

In [49]:
pi_t = cp.Variable(pred.shape[1])

objective = cp.Maximize(
    pred[0] @ pi_t -
    cp.quad_form(pi_t, Omega) -
    gamma * cp.norm(pi_t, 1)
)

objective # cvxpy проверил и говорит, что задача concave

Maximize(Expression(CONCAVE, UNKNOWN, (1, 1)))

In [50]:
cp.Problem(objective).solve()

0.45008068834266435

In [51]:
pi_t.value

array([ 1.57942466e-21,  8.46907744e+00, -8.32919392e-22,  4.68450576e-22,
       -1.98622872e-01, -1.90055041e+00, -1.02172722e-22,  7.05994196e-22,
        2.04096356e+00, -2.82280157e+00,  1.61320433e+00, -2.63921215e-21,
       -3.41401160e+00, -1.75265939e-21,  2.91228126e+00,  1.31637858e+01,
        2.59319297e-21,  4.86795951e+00,  3.93783458e-21, -2.27531951e-21])

### Part 2

At $t=2$ you already know $\pi_1$. At this moment, you have access to $p_2$ and have to maximize:
$$
p_2\pi_2 - \pi_2^T\Omega\pi_2 - \gamma \sum_i |\pi_{2, i} - \pi_{1, i}|\longrightarrow \max
$$
Repeat the process until you reach the end of the time series. The corresponding $\pi_{ti}$ should be stored as a file: this will be you 1st result in this problem. 

In [61]:
def greedy_pi(pred, Omega, gamma):
    pi_value = np.zeros(pred.shape)
    pi_t = cp.Variable(pred.shape[1])

    for t in range(pred.shape[0]):
        if t == 0:
            pi_prev_value = np.zeros(pi_t.shape)
        else:
            pi_prev_value = pi_value[t - 1]

        objective = cp.Maximize(
            pred[t] @ pi_t -
            cp.quad_form(pi_t, Omega) -
            gamma * cp.norm(pi_t - pi_prev_value, 1)
        )
        cp.Problem(objective).solve()
        pi_value[t] = pi_t.value

    return pi_value

In [84]:
greedy_pi_value = greedy_pi(pred, Omega, gamma)
greedy_pi_value

array([[ 1.57942466e-21,  8.46907744e+00, -8.32919392e-22, ...,
         4.86795951e+00,  3.93783458e-21, -2.27531951e-21],
       [ 1.69351237e-01,  1.02856449e+01, -1.57256530e-21, ...,
         4.86795951e+00,  4.07699385e-01, -2.62447167e-21],
       [ 7.26708910e-01,  1.02856449e+01, -1.50220112e-21, ...,
         4.86795951e+00,  4.07699385e-01, -1.11387329e-01],
       ...,
       [ 2.83454931e+00, -2.95594178e+00,  7.97091775e+00, ...,
         6.21879847e-01,  1.72797980e-01,  7.81929680e-01],
       [ 3.68255271e+00, -6.42039963e+00,  7.97091775e+00, ...,
         6.21879847e-01,  1.72797980e-01,  7.81929680e-01],
       [ 3.68255271e+00, -6.42039963e+00,  7.97091775e+00, ...,
         6.21879847e-01,  1.72797980e-01,  7.81929680e-01]])

In [85]:
np.save('greedy_pi.npy', greedy_pi_value)

### Part 3

considering the case of $\gamma=0$: in this case, optimization of $G$ can be done directly. Make sure the result of such direct computation coincides with $\texttt{cvxpy}$ result. The corresponding $\pi_{ti}$ should be stored as a file: this will be your 2nd result in this problem.

Решение

При $\gamma = 0$:
$$
G = \sum_{t} \left[p_t \cdot\pi_{t} - \pi_t^T \Omega \pi_t \right]
$$

Производная по $\pi_t$:
$$
\frac{\partial G}{\partial \pi_t} = p_t - 2 \Omega \pi_t = \vec 0
$$
Следовательно,
$$
\pi_t = \frac{1}{2} \Omega^{-1} p_t
$$

In [76]:
def loseless_pi(pred, Omega):
    return 0.5 * np.linalg.solve([Omega], pred)

In [86]:
loseless_pi_value = loseless_pi(pred, Omega)

In [89]:
greedy_pi_0_value = greedy_pi(pred, Omega, 0)

In [91]:
# Ответы получились разные
np.isclose(loseless_pi_value, greedy_pi_0_value).all()

False

In [92]:
# Но разница между ними мала
np.abs(loseless_pi_value - greedy_pi_0_value).max()

0.009108310576557144

In [93]:
np.save('loseless_pi.npy', loseless_pi_value)

### Part 4

Compute expected gain/costs (the 1st and the 3rd terms in Eq. (12) over the full period for two trading strategies computed above (note that the trading costs are present even if you decided to optimize at $\gamma=0$).

In [98]:
def gain(pi, pred):
    return np.sum(pi * pred)

In [119]:
def risk(pi, Omega):
    return np.einsum('ti, ij, tj -> ', pi, Omega, pi)

In [107]:
def cost(pi, gamma):
    deltas = np.diff(pi, axis=0)
    return np.sum(np.linalg.norm(deltas, ord=1, axis=1)) * gamma

In [124]:
gain(greedy_pi_value, pred), cost(greedy_pi_value, gamma)

(897.2986739223626, 35.08880299488907)

In [125]:
gain(loseless_pi_value, pred), cost(loseless_pi_value, gamma)

(1121.4631963361019, 177.15754952691802)

In [126]:
def regret(pi, pred, Omega, gamma):
    return gain(pi, pred) - risk(pi, Omega) - cost(pi, gamma)

In [127]:
regret(greedy_pi_value, pred, Omega, gamma)

414.18677080743606

In [128]:
regret(loseless_pi_value, pred, Omega, gamma)

383.5740486411329